### Importing libs

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn import linear_model
import networkx as nx

# Question 1: Propensity score matching

## Loading the data
We load the data from the provided csv file. We observe that the categorical variables are encoded using dummy varibales, i.e. binary variables. We are going to stick with this encoding as it is advantegous for the regression task that we are going to perform later.

In [ ]:
lalonde_data = pd.read_csv("lalonde.csv")
lalonde_data.head()

In [ ]:
lalonde_data.dtypes

## 1. A naive analysis

To begin with, we compare the distribution of the outcome variable (re78) between the two groups in a naive way, that is we ignore the other features in this analysis.

We use the following methods for the comparison:

A. plots:
1. boxplots
2. histograms
3. QQ plots

B. numbers:
1. summary of descriptive statistics
2. ------ test ----- TODO

As all the parts of the analysis of numerical features  also in the subsequent tasks will be based on this methods we define helping functions encapsulating this functionality:

In [ ]:
treat = {0: 'treated', 1: 'control'}

def draw_box(df, col):
    """Draw a box plot for the values of the specified column for each of the two groups"""
    df.boxplot(by='treat', column=col, figsize = [10, 7])

def draw_hist(df, col):
    """Draw histograms and kernel density estimation plots for the specified column.
    Two plots are created for the two groups but displayed in two overlapping layers for comparison.
    """
    fig_hist, axs_hist = plt.subplots(nrows=2)
    df.groupby("treat")[col].plot(kind='kde', ax=axs_hist[1])
    axs_hist[1].set_xlabel(col)
    groups = df.groupby("treat")[col]
    for k, v in groups:
        v.hist(label=treat[k], alpha=.75, ax=axs_hist[0], bins=15, range=[df[col].min(), df[col].max()], figsize = [10, 7])
    axs_hist[0].legend()
    axs_hist[0].set_ylabel('number of participants')
    

def draw_qqs(df, col):
    """Draw a QQ plot for both groups to find out more about the distribution of the data.
    NB: A comparison to a normal distribution with fitted parameters is performed."""
    fig_qq, axs_qq = plt.subplots(nrows=2, figsize=(10, 20))
    stats.probplot(df[df.treat == 0][col], dist="norm", plot=axs_qq[0])
    axs_qq[0].set_title('Control Group')
    stats.probplot(df[df.treat == 1][col], dist="norm", plot=axs_qq[1])
    axs_qq[1].set_title('Treated Group')
    plt.show()
    
def get_summary(df, col):
    """Print summary statistics for both groups."""
    print('Control group')
    print('================================================')
    print(df[df.treat == 0][col].describe())
    print('Treated group')
    print('================================================')
    print(df[df.treat == 1][col].describe())

def test(df, col):
    """TODO""" #can same test be used everywhere???? better to perform test after interpreting plot????
    # better not to always display all of them but chose????? incrementally
    #TODO use different test - data not normally distributed, willcoxon gives error
    print(stats.ttest_ind(df[df.treat == 0][col], df[df.treat == 1][col]))

Furthermore we define a funciton that invokes all those functions for a numerical feature and prints the output in a structured way.

In [ ]:
def analyse_numeric(df, col):    
    """Perform analysis for a numerical feature and pretty print result."""
    print('=================================================================================')
    print('                                  ', col)
    print('=================================================================================')
    draw_box(df, col)
    draw_hist(df, col)
    draw_qqs(df, col)
    print()
    get_summary(df, col)
    print()
    test(df, col)
    print()
    print()

Let us now analyse the distribution of the 're78' feature in the two groups. For the interpretation see below.

In [ ]:
analyse_numeric(lalonde_data, 're78')

### **(Naive) Conclusions:** .... TODO...

## 2. A closer look at the data

In order to get a better feeling for whether our naive analysis was appropriate we also compare the distributions of the other covariates. To begin with, we interpret the numerical features in using the same strategy as in Part 1.

### Analysis of numerical columns

In [ ]:
for col in ['age', 'educ', 're74', 're75']:
    analyse_numeric(lalonde_data, col)

#### **Interpretation:** .... TODO...

### Analysis of categorical columns

It remains to take a closer look at the categorical variables. As the methods used above are not appropriate for categorical variables , we need new functions for the analysis (For example, there is no point in drawing histograms for categrical variables, the thing to use here are bar plots.):

In [ ]:
def draw_bar(df, col):
    '''Draw a bar plot of the number of values in each category for the two groups'''
    df_grouped = df.groupby(['treat', col])[col].count()
    df_grouped = df_grouped.unstack()
    pl = df_grouped.plot(kind='bar', figsize=[5,5])
    pl.set_title(col)
    pl.set_ylabel('number of participants')
    pl.set_xlabel('group')
    plt.show()


In [ ]:
#TODO add more analysis....

For the race feature some preprocessing is needed in order to be able to draw a meaningful bar plot. Especially, the data frame does not contain a column for white people. We assume that individuals are white in case they are neither black nor hispanic.

In [ ]:
def plot_race(df):
    '''Draw a bar plot for the race feature.'''
    df['white'] = (~(df['black'].astype(bool) | df['hispan'].astype(bool))).astype(int)
    race = df[['white', 'black', 'hispan']].stack()
    df.drop('white', 1)
    race = pd.Series(pd.Categorical(race[race!=0].index.get_level_values(1)))
    race_group = pd.concat([df.treat, race], axis=1, keys=['treat', 'race'])
    draw_bar(race_group, 'race')

For the significance test we use Fishers exact test --- TODO justification!

In [ ]:
def test_fisher(df, col):
    '''Perform fishers exact test'''
    print('Fisher exact test')
    print('================================================')
    #compute contingency table
    df['neg'] = df[col].apply(lambda x: 1-x)
    table = df.groupby(df.treat)[col, 'neg'].sum()
    df.drop('neg', axis=1)
    #perform test
    print(stats.fisher_exact(table))

In [ ]:
def analyse_categoric(df, col=None):    
    """Perform analysis for a categorical feature and pretty print result."""
    print('=================================================================================')
    print('                                  ', col)
    print('=================================================================================')
    if(col == None):
        plot_race(df)
        print()
        print()
    else:
        draw_bar(df, col)
        print()
        get_summary(df, col)
        print()
        test_fisher(df, col)
        print()
        print()

Now we can use those functions for our analysis:

In [ ]:
for col in ['married', 'nodegree']: 
    analyse_categoric(lalonde_data, col)

analyse_categoric(lalonde_data)

**Conclusions**:

## 3. A propensity score model

In [ ]:
logistic = linear_model.LogisticRegression()

logistic = logistic.fit(lalonde_data.drop(lalonde_data.columns[[0, 1, -1]], axis=1), lalonde_data.treat)
propensity_scores = logistic.predict_proba(lalonde_data.drop(lalonde_data.columns[[0, 1, -1]], axis=1))

print(logistic.classes_)
print(propensity_scores[:5])

In [ ]:
lalonde_data['propensity'] = propensity_scores[:, 1]

## 4. Balancing the dataset via matching

1:1, without replacement
optimal matching

In [ ]:
graph_data = pd.merge(lalonde_data.reset_index()[lalonde_data.treat == 1].assign(key=0), lalonde_data.reset_index()[lalonde_data.treat == 0].assign(key=0), on='key').drop('key', axis=1)
#print(graph_data.head())
graph_data['weight'] = 1 - np.abs(graph_data['propensity_x'] - graph_data['propensity_y'])
G = nx.Graph()
G.add_weighted_edges_from(zip(graph_data.index_x, graph_data.index_y, graph_data.weight))
matching = nx.max_weight_matching(G)

In [ ]:
len(matching) == 2 * len(lalonde_data[lalonde_data.treat == 1])

In [ ]:
#todo remove :-)
for key, value in matching.items():
    if(matching[value]!=key):
        print('sad storry')
print('juhu')

In [ ]:
lalonde_data_balanced = lalonde_data.iloc[list(matching.keys())]
lalonde_data_balanced.head()

compare outcomes: see 1 and 2 + techniques from paper

In [ ]:
fig, axs = plt.subplots(nrows=2)
lalonde_data_balanced.groupby("treat").re78.plot(kind='kde', ax=axs[1])
lalonde_data_balanced.groupby("treat").re78.hist(alpha=0.4, ax=axs[0], range=[lalonde_data_balanced.re78.min(), lalonde_data_balanced.re78.max()])

## Perform same analysis as in 1 and 2 

In [ ]:
for col in ['age', 'educ', 're74', 're75', 're78']:
    analyse_numeric(lalonde_data_balanced, col)

In [ ]:
for col in ['married', 'nodegree']: 
    analyse_categoric(lalonde_data_balanced, col)

analyse_categoric(lalonde_data_balanced)

In [ ]:
#todo add test for categoric
#it somehow would make more sense for race ro see ratios...

## 5. Balancing the groups further

identify "the problematic feature": if different in this feature: add very high penalty to weight, oder: direkt in join auf equality filtern :P

problenatic features: age, race

Further balance for race:

In [ ]:
graph_data = pd.merge(lalonde_data.reset_index()[lalonde_data.treat == 1], lalonde_data.reset_index()[lalonde_data.treat == 0], on='age')
#print(graph_data.head())
graph_data['weight'] = 1 - np.abs(graph_data['propensity_x'] - graph_data['propensity_y'])
G = nx.Graph()
G.add_weighted_edges_from(zip(graph_data.index_x, graph_data.index_y, graph_data.weight))
matching = nx.max_weight_matching(G)

In [ ]:
lalonde_data_balanced = lalonde_data.iloc[list(matching.keys())]
lalonde_data_balanced.head()

### Analysis

In [ ]:
for col in ['age', 'educ', 're74', 're75', 're78']:
    analyse_numeric(lalonde_data_balanced, col)

In [ ]:
for col in ['married', 'nodegree']: 
    analyse_categoric(lalonde_data_balanced, col)

analyse_categoric(lalonde_data_balanced)

In [ ]:
#todo add test for categoric

## 6. A less naive analysis

Remark that sensitivity analysis would be good!!

In [ ]:
analyse_numeric(..., 're78')